In [1]:
import os
from prepare import PrepareDropletSubstrate

In [2]:
labels_surface = ['100r','110r','111r','100m','110m','111m']

In [3]:
labels_droplets = ['000','020','040','060','080','100']

In [4]:
for ls in labels_surface :
    os.system("mkdir "+ls)
    for ld in labels_droplets:
        os.system("mkdir "+ls+"/"+ld)

mkdir: cannot create directory ‘100r’: File exists
mkdir: cannot create directory ‘100r/000’: File exists
mkdir: cannot create directory ‘100r/020’: File exists
mkdir: cannot create directory ‘100r/040’: File exists
mkdir: cannot create directory ‘100r/060’: File exists
mkdir: cannot create directory ‘100r/080’: File exists
mkdir: cannot create directory ‘100r/100’: File exists
mkdir: cannot create directory ‘110r’: File exists
mkdir: cannot create directory ‘110r/000’: File exists
mkdir: cannot create directory ‘110r/020’: File exists
mkdir: cannot create directory ‘110r/040’: File exists
mkdir: cannot create directory ‘110r/060’: File exists
mkdir: cannot create directory ‘110r/080’: File exists
mkdir: cannot create directory ‘110r/100’: File exists
mkdir: cannot create directory ‘111r’: File exists
mkdir: cannot create directory ‘111r/000’: File exists
mkdir: cannot create directory ‘111r/020’: File exists
mkdir: cannot create directory ‘111r/040’: File exists
mkdir: cannot create d

In [5]:
!ls

100m  111r		 OldRuns	    system-preparation.ipynb
100r  droplet_test.data  prepare.py	    test.lammps
110m  droplet_test.pdb	 __pycache__	    test.py
110r  launch-runs.ipynb  surface_test.data  test-wetting
111m  log.lammps	 surface_test.pdb   wetting-pp.ipynb


In [6]:
# Set shift_z to a different number and/or define a shift (new mod) for the substrate too
scale_z = 1.0
shift_z_droplet = 0
gap_z = 5

# Better no to result to manual tuning!
"""
shift_z_substrate = dict()
shift_z_substrate['100'] = 20
shift_z_substrate['110'] = 32.5
shift_z_substrate['111'] = 45
"""

"\nshift_z_substrate = dict()\nshift_z_substrate['100'] = 20\nshift_z_substrate['110'] = 32.5\nshift_z_substrate['111'] = 45\n"

In [7]:
# Input and output files
"""
in_file_droplet = "../ti-al-droplets/droplets/020/molten_AlTi_droplet_eq.data"
in_file_surface = "../w-mo-alloy/substrate-bcc100/annealed_surface_W-Mo.data"
out_file_droplet = "droplet-test.data"
out_file_surface = "surface-test.data"
"""

'\nin_file_droplet = "../ti-al-droplets/droplets/020/molten_AlTi_droplet_eq.data"\nin_file_surface = "../w-mo-alloy/substrate-bcc100/annealed_surface_W-Mo.data"\nout_file_droplet = "droplet-test.data"\nout_file_surface = "surface-test.data"\n'

In [8]:
"""
PrepareDropletSubstrate(in_file_droplet,in_file_surface,scale_z,shift_z_droplet,
                        shift_z_substrate,out_file_droplet,out_file_surface)
"""

'\nPrepareDropletSubstrate(in_file_droplet,in_file_surface,scale_z,shift_z_droplet,\n                        shift_z_substrate,out_file_droplet,out_file_surface)\n'

In [9]:
in_folder_sub = "/home/michele/Stories/mdmc-slabs/WMo/eam-zhou/BCC-large/"

for ls in labels_surface :
    for ld in labels_droplets:
        tag_s = ls[0:3]
        annealed = ls[3]
        in_file_droplet = "../ti-al-droplets/droplets/"+ld+"/molten_AlTi_droplet_eq.data"
        # Old code
        """
        if annealed=='a' :
            in_file_surface = "../w-mo-alloy/substrate-bcc"+tag_s+"/annealed_surface_W-Mo.data"
        elif annealed=='r' :
            in_file_surface = "../w-mo-alloy/substrate-bcc"+tag_s+"/system_w_mo_min.data"
        """
        if annealed=='r' :
            in_file_surface = in_folder_sub+tag_s+"/WMo_"+tag_s+"_eq.data"
        elif annealed=='m' :
            in_file_surface = in_folder_sub+tag_s+"/WMo_"+tag_s+"_mdmc.data"
        out_file_droplet = ls+'/'+ld+"/droplet.data"
        out_file_surface = ls+'/'+ld+"/surface.data"
        PrepareDropletSubstrate(in_file_droplet,in_file_surface,scale_z,shift_z_droplet,
                                gap_z=gap_z,out_file_droplet=out_file_droplet,out_file_surface=out_file_surface)

In [10]:
import lammps

In [11]:
for ls in labels_surface :
    for ld in labels_droplets:
        lmp = lammps.lammps()
        read_data1 = ls+'/'+ld+'/droplet.data'
        read_data2 = ls+'/'+ld+'/surface.data'
        write_data = ls+'/'+ld+'/combined.data'
        lmp.commands_list([
            'units metal',
            'atom_style atomic',
            'pair_style eam/alloy',
            'neighbor 2.0 bin',
            'neigh_modify check yes',
            'boundary p p p'
            ])
        c1 = "read_data \""+read_data1+"\" extra/atom/types 2"
        c2 = "read_data \""+read_data2+"\" add append offset 2 0 0 0 0 "
        lmp.command(c1)
        lmp.command(c2)
        lmp.commands_list([
            'pair_coeff * * ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy Al Ti W Mo',
            'group liquid type 1 2',
            'group solid type 3 4',
            'group liquid type 1 2',
            'group solid type 3 4'
            ])
        c3 = "write_data \""+write_data+"\" "
        lmp.command(c3)

[denerg33012:548018] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.denerg33012.1001/jf.0/1968635904/shared_mem_cuda_pool.denerg33012 could be created.
[denerg33012:548018] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


LAMMPS (29 Aug 2024)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  orthogonal box = (0 0 0) to (251.83029 251.80836 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.035 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (251.83029 251.80836 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  128000 atoms
  reading velocities ...
  128000 velocities
  read_data CPU = 0.357 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy with DATE: 2018-03-30
12195 atoms in group liquid
128000 atoms in group solid
12195 atoms in group liquid
128000 atoms in group solid
System init for write_data ...
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = yes
  max neighbors/atom: 2000, page size: 100000
  mast

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (251.96661 256.02293 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.033 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (251.96661 256.02293 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  127680 atoms
  reading velocities ...
  127680 velocities
  read_data CPU = 0.347 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgC

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (251.96661 256.02293 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.034 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (251.96661 256.02293 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  127680 atoms
  reading velocities ...
  127680 velocities
  read_data CPU = 0.351 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgC

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (255.34323 253.43548 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.034 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (255.34323 253.43548 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  134064 atoms
  reading velocities ...
  134064 velocities
  read_data CPU = 0.374 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgC

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (252.07634 252.11604 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.034 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (252.07634 252.11604 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  128000 atoms
  reading velocities ...
  128000 velocities
  read_data CPU = 0.348 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgC

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (252.2377 256.06873 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.033 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (252.2377 256.06873 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  127680 atoms
  reading velocities ...
  127680 velocities
  read_data CPU = 0.339 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgCoT

OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Your simulation uses code contributions which should be cited:
- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419
The log file lists these citations in BibTeX format.

CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE

Total wall time: 0:00:00
Reading data file ...
  orthogonal box = (0 0 0) to (255.35382 253.44991 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  12195 atoms
  reading velocities ...
  12195 velocities
  read_data CPU = 0.033 seconds
Reading data file ...
  orthogonal box = (0 0 0) to (255.35382 253.44991 166.72803)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  134064 atoms
  reading velocities ...
  134064 velocities
  read_data CPU = 0.353 seconds
Reading eam/alloy potential file ../potential-files/eam/CuAgAuNiPdPtAlPbFeMoTaWMgC